In [2]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import numpy as np
import os
import argparse
import utils
from fer import FER2013
from torch.autograd import Variable
from models import *
from torchvision import models
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import csv

In [3]:
model = models.squeezenet1_1(pretrained=True)
model.classifier[1] = nn.Conv2d(512, 7, kernel_size=(1,1), stride=(1,1))  

F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
class Opt:
    def __init__(self):
        self.model = 'SqueezeNet'  
        self.dataset = 'FER2013'
        self.bs = 128
        self.lr = 0.01
        self.resume = False

opt = Opt()

In [5]:
use_cuda = torch.cuda.is_available()
best_Testing_acc = 0 
best_Testing_acc_epoch = 0
best_Validation_acc = 0 
best_Validation_acc_epoch = 0
start_epoch = 0  

learning_rate_decay_start = 80
learning_rate_decay_every = 5
learning_rate_decay_rate = 0.9

cut_size = 44
total_epoch = 100

path = os.path.join(opt.dataset + '_' + opt.model)

# Lists to save metrics
train_acc_list = []
train_loss_list = []
testing_acc_list = []
testing_loss_list = []
validation_acc_list = []
validation_loss_list = []
train_precision_list, train_recall_list, train_f1_list, train_auc_list = [], [], [], []
testing_precision_list, testing_recall_list, testing_f1_list, testing_auc_list = [], [], [], []
validation_precision_list, validation_recall_list, validation_f1_list, validation_auc_list = [], [], [], []

In [14]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(44),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.TenCrop(cut_size),
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
])

trainset = FER2013(split='Training', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=opt.bs, shuffle=True, num_workers=0)
Testingset = FER2013(split='Testing', transform=transform_test)
Testingloader = torch.utils.data.DataLoader(Testingset, batch_size=opt.bs, shuffle=False, num_workers=0)
Validationset = FER2013(split='Validation', transform=transform_test)
Validationloader = torch.utils.data.DataLoader(Validationset, batch_size=opt.bs, shuffle=False, num_workers=0)

==> Preparing data..


In [15]:
print('==> Building model..')
net = models.squeezenet1_1(pretrained=True)
net.classifier[1] = nn.Conv2d(512, 7, kernel_size=(1,1), stride=(1,1))

if not os.path.isdir(path):
    print("No checkpoint directory found. Starting training from scratch.")
    opt.resume = False

if opt.resume:
    print('==> Resuming from checkpoint..')
    checkpoint = torch.load(os.path.join(path, 'Validation_model.t7'), map_location=torch.device('cpu'))
    net.load_state_dict(checkpoint['net'])
    best_Testing_acc = checkpoint['best_Testing_acc']
    best_Validation_acc = checkpoint['best_Validation_acc']
    best_Testing_acc_epoch = checkpoint['best_Testing_acc_epoch']
    best_Validation_acc_epoch = checkpoint['best_Validation_acc_epoch']
    start_epoch = checkpoint['best_Validation_acc_epoch'] + 1
else:
    print('==> Starting training from scratch.')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=opt.lr, momentum=0.9, weight_decay=5e-4)

==> Building model..
==> Starting training from scratch.


In [16]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    global Train_acc
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    if epoch > learning_rate_decay_start and learning_rate_decay_start >= 0:
        frac = (epoch - learning_rate_decay_start) // learning_rate_decay_every
        decay_factor = learning_rate_decay_rate ** frac
        current_lr = opt.lr * decay_factor
        utils.set_lr(optimizer, current_lr)
    else:
        current_lr = opt.lr
    print('learning_rate: %s' % str(current_lr))

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs, targets
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        utils.clip_gradient(optimizer, 0.1)
        optimizer.step()
        train_loss += loss.item()  
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).sum().item()

        utils.progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                           % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    Train_acc = 100.*correct/total
    train_acc_list.append(Train_acc)
    train_loss_list.append(train_loss / len(trainloader))

    outputs_np = outputs.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()
    predicted_np = np.argmax(outputs_np, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(targets_np, predicted_np, average='weighted')
    try:
        auc = roc_auc_score(targets_np, outputs_np, multi_class='ovr')
    except ValueError:
        auc = 0.0  # Handle cases where AUC cannot be calculated
    
    train_precision_list.append(precision)
    train_recall_list.append(recall)
    train_f1_list.append(f1)
    train_auc_list.append(auc)

In [17]:
def Testing(epoch):
    global Testing_acc
    global best_Testing_acc
    global best_Testing_acc_epoch
    net.eval()
    Testing_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(Testingloader):
        bs, ncrops, c, h, w = np.shape(inputs)
        inputs = inputs.view(-1, c, h, w)
        inputs, targets = inputs, targets
        with torch.no_grad():
            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)
            loss = criterion(outputs_avg, targets)
            Testing_loss += loss.item()
            _, predicted = torch.max(outputs_avg.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).sum().item()

        utils.progress_bar(batch_idx, len(Testingloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                           % (Testing_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    Testing_acc = 100.*correct/total
    if Testing_acc > best_Testing_acc:
        print('Saving..')
        print("best_Testing_acc: %0.3f" % Testing_acc)
        state = {
            'net': net.state_dict(),
            'acc': Testing_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Testing_model.t7'))
        best_Testing_acc = Testing_acc
        best_Testing_acc_epoch = epoch

    testing_acc_list.append(Testing_acc)
    testing_loss_list.append(Testing_loss / len(Testingloader))
    outputs_np = outputs_avg.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()
    predicted_np = np.argmax(outputs_np, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(targets_np, predicted_np, average='weighted')
    try:
        auc = roc_auc_score(targets_np, outputs_np, multi_class='ovr')
    except ValueError:
        auc = 0.0
    
    testing_precision_list.append(precision)
    testing_recall_list.append(recall)
    testing_f1_list.append(f1)
    testing_auc_list.append(auc)


In [18]:
def Validation(epoch):
    global Validation_acc
    global best_Validation_acc
    global best_Validation_acc_epoch
    net.eval()
    Validation_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(Validationloader):
            bs, ncrops, c, h, w = np.shape(inputs)
            inputs = inputs.view(-1, c, h, w)
            inputs, targets = inputs, targets

            outputs = net(inputs)
            outputs_avg = outputs.view(bs, ncrops, -1).mean(1)
            loss = criterion(outputs_avg, targets)
            Validation_loss += loss.item()

            _, predicted = torch.max(outputs_avg.data, 1)
            correct += predicted.eq(targets.data).sum().item()
            total += targets.size(0)

            utils.progress_bar(batch_idx, len(Validationloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                               % (Validation_loss / (batch_idx + 1), 100. * correct / total, correct, total))

    Validation_acc = 100. * correct / total
    if Validation_acc > best_Validation_acc:
        print('Saving..')
        print("best_Validation_acc: %0.3f" % Validation_acc)
        state = {
            'net': net.state_dict(),
            'acc': Validation_acc,
            'epoch': epoch,
        }
        if not os.path.isdir(path):
            os.mkdir(path)
        torch.save(state, os.path.join(path, 'Validation_model.t7'))
        best_Validation_acc = Validation_acc
        best_Validation_acc_epoch = epoch

    validation_acc_list.append(Validation_acc)
    validation_loss_list.append(Validation_loss / len(Validationloader))
    outputs_np = outputs_avg.detach().cpu().numpy()
    targets_np = targets.detach().cpu().numpy()
    predicted_np = np.argmax(outputs_np, axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(targets_np, predicted_np, average='weighted')
    try:
        auc = roc_auc_score(targets_np, outputs_np, multi_class='ovr')
    except ValueError:
        auc = 0.0
    
    validation_precision_list.append(precision)
    validation_recall_list.append(recall)
    validation_f1_list.append(f1)
    validation_auc_list.append(auc)


In [19]:
def save_metrics():
    metrics_file_path = os.path.join(path, "metrics.csv")
    
    # Combine all metrics into a single table
    metrics = zip(
        range(1, len(train_acc_list) + 1),  # Epoch numbers
        train_acc_list, train_loss_list, train_precision_list, train_recall_list, train_f1_list, train_auc_list,
        testing_acc_list, testing_loss_list, testing_precision_list, testing_recall_list, testing_f1_list, testing_auc_list,
        validation_acc_list, validation_loss_list, validation_precision_list, validation_recall_list, validation_f1_list, validation_auc_list,
    )
    
    # Write metrics to the CSV file
    with open(metrics_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            "Epoch", 
            "Train Accuracy", "Train Loss", "Train Precision", "Train Recall", "Train F1", "Train AUC",
            "Testing Accuracy", "Testing Loss", "Testing Precision", "Testing Recall", "Testing F1", "Testing AUC",
            "Validation Accuracy", "Validation Loss", "Validation Precision", "Validation Recall", "Validation F1", "Validation AUC",
        ])
        for row in metrics:
            writer.writerow(row)
    
    print(f"Metrics saved to {metrics_file_path}")

In [20]:
for epoch in range(start_epoch, total_epoch):
    train(epoch)
    Testing(epoch)
    Validation(epoch)
    save_metrics()


Epoch: 0
learning_rate: 0.01
 [=============================>] | Loss: 1.903 | Acc: 24.525% (7041/28709)     225/225 


F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 [============================>.] | Loss: 1.796 | Acc: 24.492% (879/3589)       29/29 
Saving..
best_Testing_acc: 24.492


F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 [============================>.] | Loss: 1.796 | Acc: 24.492% (879/3589)       29/29 
Saving..
best_Validation_acc: 24.492
Metrics saved to FER2013_SqueezeNet\metrics.csv

Epoch: 1
learning_rate: 0.01


...

F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 [=============================>] | Loss: 0.826 | Acc: 69.159% (19855/28709)    225/225 
 [============================>.] | Loss: 0.935 | Acc: 64.865% (2328/3589)      29/29 


F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 [============================>.] | Loss: 0.935 | Acc: 64.865% (2328/3589)      29/29 
Metrics saved to FER2013_SqueezeNet\metrics.csv


F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
F:\FER-2013 Project\facialexpressionrecogination\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
